<a href="https://colab.research.google.com/github/Judy-Choi/Tensorflow_Certificate/blob/main/8_horses_vs_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal Metric
- val_loss : 0.028
- val_acc : 0.99

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 1. 데이터셋 준비하기

In [2]:
dataset_name = 'horses_or_humans'
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLEJF8H/horses_or_humans-train.tfrecord*..…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/horses_or_humans/3.0.0.incompleteLEJF8H/horses_or_humans-test.tfrecord*...…

Dataset horses_or_humans downloaded and prepared to /root/tensorflow_datasets/horses_or_humans/3.0.0. Subsequent calls will reuse this data.


# 2. 데이터 프리프로세싱

In [ ]:
def preprocess(features):
    image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
    image = tf.image.resize(image, size=(300, 300))
    return image, label


batch_size=32
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)


# 4. 모델설계하기

In [ ]:

model = Sequential([
    Conv2D(64, (3, 3), input_shape=(300, 300, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax'),
])
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# 5. 모델 모니터링하기

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint_path = 'checkpoint.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, 
        save_best_only=True, 
        save_weights_only=True, 
        monitor='val_loss', 
        verbose=1)


# 6. 모델 학습하기

In [ ]:
history = model.fit(train_data, 
       validation_data=(valid_data),
       epochs=50, 
       callbacks=[checkpoint, early_stopping])

model.load_weights(checkpoint_path)
model.save("horses-humans-1.h5")

Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 0.6947 - accuracy: 0.5474
Epoch 1: val_loss improved from inf to 0.65976, saving model to checkpoint.ckpt
26/26 [==============================] - 20s 249ms/step - loss: 0.6947 - accuracy: 0.5474 - val_loss: 0.6598 - val_accuracy: 0.5317
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.5587 - accuracy: 0.7372
Epoch 2: val_loss improved from 0.65976 to 0.47504, saving model to checkpoint.ckpt
26/26 [==============================] - 5s 187ms/step - loss: 0.5587 - accuracy: 0.7372 - val_loss: 0.4750 - val_accuracy: 0.8098
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.3313 - accuracy: 0.8783
Epoch 3: val_loss improved from 0.47504 to 0.23770, saving model to checkpoint.ckpt
26/26 [==============================] - 5s 189ms/step - loss: 0.3313 - accuracy: 0.8783 - val_loss: 0.2377 - val_accuracy: 0.9024
Epoch 4/50
26/26 [==============================] - ETA: 0s - loss: 0.2700 -

In [ ]:
print(history.history["val_loss"])

[0.5121221542358398, 1.0606327056884766, 0.10092807561159134, 0.04546903446316719, 0.024433353915810585, 0.006289195269346237, 0.012726910412311554, 0.002693785121664405, 0.006885387469083071, 0.0008683903724886477, 0.002668134868144989, 0.016847286373376846, 0.000588846392929554, 0.014058411121368408, 0.003168781753629446, 0.0020299239549785852, 0.0008267945377156138, 0.07081251591444016, 0.002800089307129383, 0.003058104310184717, 0.021328791975975037, 0.00038117365329526365, 3.556426236173138e-05, 1.9391569367144257e-05, 0.037739552557468414, 0.0005001993267796934, 0.00017365584790240973, 0.01685124635696411, 9.050055086845532e-05, 4.0760260162642226e-05, 4.735690890811384e-05, 0.004066681023687124, 8.62730885273777e-05, 1.1097219612565823e-05, 5.632381544273812e-06, 0.0009556927252560854, 9.39124365686439e-05, 3.129955439362675e-05, 0.0003076934372074902, 3.462984022917226e-05, 1.484452127442637e-06, 6.587563257198781e-05, 5.669323400070425e-06, 4.535517291515134e-06, 3.90184368370

In [ ]:
print(history.history["val_accuracy"])

[0.8292682766914368, 0.497560977935791, 0.9609755873680115, 0.9902439117431641, 0.9902439117431641, 0.995121955871582, 0.995121955871582, 1.0, 0.995121955871582, 1.0, 1.0, 0.995121955871582, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9756097793579102, 1.0, 1.0, 0.995121955871582, 1.0, 1.0, 1.0, 0.995121955871582, 1.0, 1.0, 0.9902439117431641, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


# Homework 1 : CNN - 0.00862

In [6]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# This task requires you to create a classifier for horses or humans using
# the provided dataset. 
#
# Please make sure your **final layer has 2 neurons, activated by softmax **
# as shown. Do not change the provided output layer, or tests may fail.
#
# IMPORTANT: Please note that 
# ** the test uses images that are 300x300 with 3 bytes color depth **
# so be sure to design your input layer to accept
# these, or the tests will fail.
#

import tensorflow_datasets as tfds
import tensorflow as tf

# import libraries
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

dataset_name = 'horses_or_humans'
# dataset, info = tfds.load(name=dataset_name, split=tfds.Split.TRAIN, with_info=True)

# split dataset
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

def preprocess(features):
    # YOUR CODE HERE
    # Normalize
    image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
    # the test uses images that are 300x300 with 3 bytes color depth
    image = tf.image.resize(image, size=(300, 300))
    return image, label

def solution_model():
    # preprocess train / valid
    # train_dataset = dataset.map(preprocess).batch(32)
    train_data = train_dataset.map(preprocess).batch(32)
    valid_data = valid_dataset.map(preprocess).batch(32)

    # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
    # model = Sequential
    model = Sequential([
        # [CNN - MaxPooling] * N + Flatten() + Dense * N
        # the test uses images that are 300x300 with 3 bytes color depth
        Conv2D(64, (3, 3), input_shape=(300, 300, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(16, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(16, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),

        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),

        # 2 NEURONS ACTIVATED BY SOFTMAX
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    # optimizer
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # checkpoint
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
            save_best_only=True, 
            save_weights_only=True, 
            monitor='val_loss', 
            verbose=1)
    
    history = model.fit(
                    train_data, 
                    validation_data=valid_data,
                    epochs=50, 
                    callbacks=[checkpoint, early_stopping])

    model.load_weights(checkpoint_path)

    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 0.6904 - accuracy: 0.5097
Epoch 1: val_loss improved from inf to 0.68786, saving model to checkpoint.ckpt
26/26 [==============================] - 23s 289ms/step - loss: 0.6904 - accuracy: 0.5097 - val_loss: 0.6879 - val_accuracy: 0.4585
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.6261 - accuracy: 0.6448
Epoch 2: val_loss improved from 0.68786 to 0.40353, saving model to checkpoint.ckpt
26/26 [==============================] - 3s 114ms/step - loss: 0.6261 - accuracy: 0.6448 - val_loss: 0.4035 - val_accuracy: 0.8976
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.3773 - accuracy: 0.8491
Epoch 3: val_loss improved from 0.40353 to 0.35815, saving model to checkpoint.ckpt
26/26 [==============================] - 3s 112ms/step - loss: 0.3773 - accuracy: 0.8491 - val_loss: 0.3582 - val_accuracy: 0.8341
Epoch 4/50
26/26 [==============================] - ETA: 0s - loss: 0.3134 -

KeyboardInterrupt: ignored

# Homework 2 : Transfer Learning (VGG16) - 0.00053

In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# This task requires you to create a classifier for horses or humans using
# the provided dataset. 
#
# Please make sure your **final layer has 2 neurons, activated by softmax **
# as shown. Do not change the provided output layer, or tests may fail.
#
# IMPORTANT: Please note that 
# ** the test uses images that are 300x300 with 3 bytes color depth **
# so be sure to design your input layer to accept
# these, or the tests will fail.
#

import tensorflow_datasets as tfds
import tensorflow as tf

# import libraries
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# VGG16
from tensorflow.keras.applications import VGG16

dataset_name = 'horses_or_humans'
# dataset, info = tfds.load(name=dataset_name, split=tfds.Split.TRAIN, with_info=True)

# split dataset
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

def preprocess(features):
    # YOUR CODE HERE
    # Normalize
    image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
    # the test uses images that are 300x300 with 3 bytes color depth
    image = tf.image.resize(image, size=(300, 300))
    return image, label

def solution_model():
    # preprocess train / valid
    # train_dataset = dataset.map(preprocess).batch(32)
    train_data = train_dataset.map(preprocess).batch(32)
    valid_data = valid_dataset.map(preprocess).batch(32)

    # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX

    # Code to define the model
    # the test uses images that are 300x300 with 3 bytes color depth
    transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
    transfer_model.trainable=False

    # model = Sequential
    model = Sequential([
        # transfer_model + Flatten() + Dense * N
        transfer_model,

        Flatten(),

        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),

        # 2 NEURONS ACTIVATED BY SOFTMAX
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    # optimizer
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # checkpoint
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    checkpoint_path = 'checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
            save_best_only=True, 
            save_weights_only=True, 
            monitor='val_loss', 
            verbose=1)
    
    history = model.fit(
                    train_data, 
                    validation_data=valid_data,
                    epochs=50, 
                    callbacks=[checkpoint, early_stopping])

    model.load_weights(checkpoint_path)

    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 1.8172 - accuracy: 0.8540
Epoch 1: val_loss improved from inf to 0.01457, saving model to checkpoint.ckpt
26/26 [==============================] - 31s 887ms/step - loss: 1.8172 - accuracy: 0.8540 - val_loss: 0.0146 - val_accuracy: 1.0000
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.6181 - accuracy: 0.9380
Epoch 2: val_loss improved from 0.01457 to 0.01074, saving model to checkpoint.ckpt
26/26 [==============================] - 9s 356ms/step - loss: 0.6181 - accuracy: 0.9380 - val_loss: 0.0107 - val_accuracy: 0.9951
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.0053 - accuracy: 0.9988
Epoch 3: val_loss did not improve from 0.01074
26/26 [==============================] - 8s 313ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.0126 - val_accuracy: 0.9902
Epoch 4/50
26/26 [==============================

# Homework 3 : ChatGPT(CNN) - 0.00038

다음 코드를 최적화해 줘,  
단, 다음 조건을 적용해 줘  
Use dropout  
Use batch normalization  
Use checkpoint with save best only, and save weight only and verbose = 1  
Use early stopping with patience = 10  
Use reduce learning rate on plateau  
set verbose = 1  
Set epochs = 50  
do not use data augmentation  
Use other methods to make val loss value lower.  


최적화 내용:

- BatchNormalization을 추가하여, 각 층의 출력값의 분산과 평균을 정규화하여 학습 안정성을 높였습니다.
- Dropout을 추가하여, overfitting을 방지하고 일반화 성능을 높였습니다.
- callbacks으로 ModelCheckpoint와 EarlyStopping, ReduceLROnPlateau을 추가하여, val_loss의 성능을 개선하고, overfitting을 방지하고, learning rate를 동적으로 변경하여 학습 안정성을 높였습니다.
- loss 함수를 'categorical_crossentropy'로 변경하여, softmax activation function과 함께 사용하도록 하였습니다.
- 학습 과정에서, val_loss가 낮아지지 않을 때, 측정값이 어느 정도 나타난 후, 학습속도를 더욱 느리게 하여 최적화 결과를 개선하도록 하였습니다.

In [13]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# This task requires you to create a classifier for horses or humans using
# the provided dataset. 
#
# Please make sure your **final layer has 2 neurons, activated by softmax **
# as shown. Do not change the provided output layer, or tests may fail.
#
# IMPORTANT: Please note that 
# ** the test uses images that are 300x300 with 3 bytes color depth **
# so be sure to design your input layer to accept
# these, or the tests will fail.
#

import tensorflow_datasets as tfds
import tensorflow as tf

# import libraries
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

dataset_name = 'horses_or_humans'
# dataset, info = tfds.load(name=dataset_name, split=tfds.Split.TRAIN, with_info=True)

# split dataset
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

def preprocess(features):
    # YOUR CODE HERE
    # Normalize
    image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
    # the test uses images that are 300x300 with 3 bytes color depth
    image = tf.image.resize(image, size=(300, 300))
    return image, label

def solution_model():
    # preprocess train / valid
    # train_dataset = dataset.map(preprocess).batch(32)
    train_data = train_dataset.map(preprocess).batch(32)
    valid_data = valid_dataset.map(preprocess).batch(32)

    # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
    # model = Sequential
    model = Sequential([
        Conv2D(64, (3, 3), input_shape=(300, 300, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Conv2D(16, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Conv2D(16, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Flatten(),
        
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])

    # optimizer
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


    checkpoint_path = 'my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
                                save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss',
                                verbose=1)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    
    history = model.fit(
                    train_data, 
                    validation_data=valid_data,
                    epochs=50, 
                    callbacks=[checkpoint, early_stopping])

    model.load_weights(checkpoint_path)

    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.6557
Epoch 1: val_loss improved from inf to 0.69054, saving model to my_checkpoint.ckpt
26/26 [==============================] - 10s 225ms/step - loss: 0.6930 - accuracy: 0.6557 - val_loss: 0.6905 - val_accuracy: 0.5659
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.3661 - accuracy: 0.8260
Epoch 2: val_loss improved from 0.69054 to 0.66884, saving model to my_checkpoint.ckpt
26/26 [==============================] - 4s 171ms/step - loss: 0.3661 - accuracy: 0.8260 - val_loss: 0.6688 - val_accuracy: 0.5659
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.2048 - accuracy: 0.9197
Epoch 3: val_loss did not improve from 0.66884
26/26 [==============================] - 4s 163ms/step - loss: 0.2048 - accuracy: 0.9197 - val_loss: 0.6692 - val_accuracy: 0.5659
Epoch 4/50
26/26 [==============================] - ETA: 0s - loss: 0.1532 - accuracy: 0.9367
Epoch 4: val_

# Homework 4 : ChatGPT(ResNet50V2) - 0.02169
다음 코드의 CNN 을 Transfer learning 으로 바꿔 줘
단, 모델은 VGG16 이외의 모델을 써 줘

ChatGPT answer:

여기서는 ResNet50V2 모델을 가져와서, include_top=False로 설정하여 마지막 fully connected layer를 제외하고 모델을 가져왔습니다. 그리고 이후에 새로운 fully connected layer를 추가해서 모델을 완성합니다. 가져온 pre-trained model의 layer들은 trainable 속성을 False로 설정하여 그대로 사용하고, 새로운 layer들만 trainable 속성을 True로 설정하여 fine-tuning을 합니다.

마지막으로 optimizer와 loss function을 설정하고 compile합니다.

In [17]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# Computer Vision with CNNs
#
# This task requires you to create a classifier for horses or humans using
# the provided dataset. 
#
# Please make sure your **final layer has 2 neurons, activated by softmax **
# as shown. Do not change the provided output layer, or tests may fail.
#
# IMPORTANT: Please note that 
# ** the test uses images that are 300x300 with 3 bytes color depth **
# so be sure to design your input layer to accept
# these, or the tests will fail.
#

import tensorflow_datasets as tfds
import tensorflow as tf

# import libraries
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.applications.resnet_v2 import ResNet50V2

dataset_name = 'horses_or_humans'
# dataset, info = tfds.load(name=dataset_name, split=tfds.Split.TRAIN, with_info=True)

# split dataset
train_dataset, info = tfds.load(name=dataset_name, split='train[:80%]', with_info=True)
valid_dataset, info = tfds.load(name=dataset_name, split='train[-20%:]', with_info=True)

def preprocess(features):
    # YOUR CODE HERE
    # Normalize
    image, label = tf.cast(features['image'], tf.float32) / 255.0, features['label']
    # the test uses images that are 300x300 with 3 bytes color depth
    image = tf.image.resize(image, size=(300, 300))
    return image, label

def solution_model():
    # preprocess train / valid
    # train_dataset = dataset.map(preprocess).batch(32)
    train_data = train_dataset.map(preprocess).batch(32)
    valid_data = valid_dataset.map(preprocess).batch(32)

    # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
    # model = Sequential

    base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(300, 300, 3))

    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])

    # optimizer
    # model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


    checkpoint_path = 'my_checkpoint.ckpt'
    checkpoint = ModelCheckpoint(checkpoint_path, 
                                save_weights_only=True, 
                                save_best_only=True, 
                                monitor='val_loss',
                                verbose=1)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    
    history = model.fit(
                    train_data, 
                    validation_data=valid_data,
                    epochs=50, 
                    callbacks=[checkpoint, early_stopping])

    model.load_weights(checkpoint_path)

    return model

# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")


Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 1.3662 - accuracy: 0.7944
Epoch 1: val_loss improved from inf to 8247.45996, saving model to my_checkpoint.ckpt
26/26 [==============================] - 47s 723ms/step - loss: 1.3662 - accuracy: 0.7944 - val_loss: 8247.4600 - val_accuracy: 0.4341
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.3854 - accuracy: 0.9136
Epoch 2: val_loss did not improve from 8247.45996
26/26 [==============================] - 15s 568ms/step - loss: 0.3854 - accuracy: 0.9136 - val_loss: 32946.4258 - val_accuracy: 0.4341
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.4737 - accuracy: 0.8443
Epoch 3: val_loss did not improve from 8247.45996
26/26 [==============================] - 14s 556ms/step - loss: 0.4737 - accuracy: 0.8443 - val_loss: 2748217.5000 - val_accuracy: 0.4341
Epoch 4/50
26/26 [==============================] - ETA: 0s - loss: 0.5528 - accuracy: 0.8954
Epoch 4: val_loss did not imp